# Meta Inc. Stock price predictions

Let's take a look to stock prices and market capitalisation as a result of how market reacts to financial statistics of the company. 
First of all we need to extract financial statistics from the web. After that we need to determine affection of this statistics to stock price and discover correlation.
Final step will be building model to predict market capitalisation for the next quorter.

Import libraries

In [3]:
import pandas as pd
!mamba install bs4==4.10.0 -y
!mamba install html5lib==1.1 -y


                  __    __    __    __
                 /  \  /  \  /  \  /  \
                /    \/    \/    \/    \
███████████████/  /██/  /██/  /██/  /████████████████████████
              /  / \   / \   / \   / \  \____
             /  /   \_/   \_/   \_/   \    o \__,
            / _/                       \_____/  `
            |/
        ███╗   ███╗ █████╗ ███╗   ███╗██████╗  █████╗
        ████╗ ████║██╔══██╗████╗ ████║██╔══██╗██╔══██╗
        ██╔████╔██║███████║██╔████╔██║██████╔╝███████║
        ██║╚██╔╝██║██╔══██║██║╚██╔╝██║██╔══██╗██╔══██║
        ██║ ╚═╝ ██║██║  ██║██║ ╚═╝ ██║██████╔╝██║  ██║
        ╚═╝     ╚═╝╚═╝  ╚═╝╚═╝     ╚═╝╚═════╝ ╚═╝  ╚═╝

        mamba (0.7.6) supported by @QuantStack

        GitHub:  https://github.com/mamba-org/mamba
        Twitter: https://twitter.com/QuantStack

█████████████████████████████████████████████████████████████


Looking for: ['bs4==4.10.0']

pkgs/main/linux-64       Using cache
pkgs/main/noarch         Using cache
pkgs/r/li

In [4]:
#Import libraries for webscrapping
import requests
from bs4 import BeautifulSoup

Set url and get request

In [5]:
meta_data_url = "https://www.macrotrends.net/stocks/charts/META/meta-platforms/income-statement"  

In [12]:
#Request to website
html_data = requests.get(meta_data_url).text

In [13]:
#Create Beautiful Soup object
souped_html = BeautifulSoup(html_data, 'html5lib')

In [14]:
# Check title of the object to make sure that everything is going well
souped_html.title

<title>Meta Platforms Income Statement 2009-2022 | META | MacroTrends</title>

In [30]:
script_text = souped_html.find_all('script')   # Find the second table with Quarterly Revenue /or use 0 for first table

In [34]:
script_text = [{"field_name":"<a href='\/stocks\/charts\/META\/meta-platforms\/revenue'>Revenue<\/a>","popup_icon":"<div class='ajax-chart' data-tipped-options=\"ajax: {data: { t: 'META', s: 'revenue', freq: 'A', statement: 'income-statement' }}\"><i style='font-size:18px; color:#337ab7;' class='fas fa-chart-bar'><\/i><\/span><\/div>","2021-12-31":"117929.00000","2020-12-31":"85965.00000","2019-12-31":"70697.00000","2018-12-31":"55838.00000","2017-12-31":"40653.00000","2016-12-31":"27638.00000","2015-12-31":"17928.00000","2014-12-31":"12466.00000","2013-12-31":"7872.00000","2012-12-31":"5089.00000","2011-12-31":"3711.00000","2010-12-31":"1974.00000","2009-12-31":"777.00000"},{"field_name":"<a href='\/stocks\/charts\/META\/meta-platforms\/cost-goods-sold'>Cost Of Goods Sold<\/a>","popup_icon":"<div class='ajax-chart' data-tipped-options=\"ajax: {data: { t: 'META', s: 'cost-goods-sold', freq: 'A', statement: 'income-statement' }}\"><i style='font-size:18px; color:#337ab7;' class='fas fa-chart-bar'><\/i><\/span><\/div>","2021-12-31":"22649.00000","2020-12-31":"16692.00000","2019-12-31":"12770.00000","2018-12-31":"9355.00000","2017-12-31":"5454.00000","2016-12-31":"3789.00000","2015-12-31":"2867.00000","2014-12-31":"2153.00000","2013-12-31":"1875.00000","2012-12-31":"1364.00000","2011-12-31":"860.00000","2010-12-31":"493.00000","2009-12-31":"223.00000"},{"field_name":"<a href='\/stocks\/charts\/META\/meta-platforms\/gross-profit'>Gross Profit<\/a>","popup_icon":"<div class='ajax-chart' data-tipped-options=\"ajax: {data: { t: 'META', s: 'gross-profit', freq: 'A', statement: 'income-statement' }}\"><i style='font-size:18px; color:#337ab7;' class='fas fa-chart-bar'><\/i><\/span><\/div>","2021-12-31":"95280.00000","2020-12-31":"69273.00000","2019-12-31":"57927.00000","2018-12-31":"46483.00000","2017-12-31":"35199.00000","2016-12-31":"23849.00000","2015-12-31":"15061.00000","2014-12-31":"10313.00000","2013-12-31":"5997.00000","2012-12-31":"3725.00000","2011-12-31":"2851.00000","2010-12-31":"1481.00000","2009-12-31":"554.00000"},{"field_name":"<a href='\/stocks\/charts\/META\/meta-platforms\/research-development-expenses'>Research And Development Expenses<\/a>","popup_icon":"<div class='ajax-chart' data-tipped-options=\"ajax: {data: { t: 'META', s: 'research-development-expenses', freq: 'A', statement: 'income-statement' }}\"><i style='font-size:18px; color:#337ab7;' class='fas fa-chart-bar'><\/i><\/span><\/div>","2021-12-31":"24655.00000","2020-12-31":"18447.00000","2019-12-31":"13600.00000","2018-12-31":"10273.00000","2017-12-31":"7754.00000","2016-12-31":"5919.00000","2015-12-31":"4816.00000","2014-12-31":"2666.00000","2013-12-31":"1415.00000","2012-12-31":"1399.00000","2011-12-31":"388.00000","2010-12-31":"144.00000","2009-12-31":"87.00000"},{"field_name":"<a href='\/stocks\/charts\/META\/meta-platforms\/selling-general-administrative-expenses'>SG&A Expenses<\/a>","popup_icon":"<div class='ajax-chart' data-tipped-options=\"ajax: {data: { t: 'META', s: 'selling-general-administrative-expenses', freq: 'A', statement: 'income-statement' }}\"><i style='font-size:18px; color:#337ab7;' class='fas fa-chart-bar'><\/i><\/span><\/div>","2021-12-31":"23872.00000","2020-12-31":"18155.00000","2019-12-31":"20341.00000","2018-12-31":"11297.00000","2017-12-31":"7242.00000","2016-12-31":"5503.00000","2015-12-31":"4020.00000","2014-12-31":"2653.00000","2013-12-31":"1778.00000","2012-12-31":"1788.00000","2011-12-31":"707.00000","2010-12-31":"305.00000","2009-12-31":"205.00000"},{"field_name":"<span style='color:#337ab7;'>Other Operating Income Or Expenses<\/span>","popup_icon":"","2021-12-31":"","2020-12-31":"","2019-12-31":"","2018-12-31":"","2017-12-31":"","2016-12-31":"","2015-12-31":"","2014-12-31":"","2013-12-31":"","2012-12-31":"","2011-12-31":"","2010-12-31":"","2009-12-31":""},{"field_name":"<a href='\/stocks\/charts\/META\/meta-platforms\/operating-expenses'>Operating Expenses<\/a>","popup_icon":"<div class='ajax-chart' data-tipped-options=\"ajax: {data: { t: 'META', s: 'operating-expenses', freq: 'A', statement: 'income-statement' }}\"><i style='font-size:18px; color:#337ab7;' class='fas fa-chart-bar'><\/i><\/span><\/div>","2021-12-31":"71176.00000","2020-12-31":"53294.00000","2019-12-31":"46711.00000","2018-12-31":"30925.00000","2017-12-31":"20450.00000","2016-12-31":"15211.00000","2015-12-31":"11703.00000","2014-12-31":"7472.00000","2013-12-31":"5068.00000","2012-12-31":"4551.00000","2011-12-31":"1955.00000","2010-12-31":"942.00000","2009-12-31":"515.00000"},{"field_name":"<a href='\/stocks\/charts\/META\/meta-platforms\/operating-income'>Operating Income<\/a>","popup_icon":"<div class='ajax-chart' data-tipped-options=\"ajax: {data: { t: 'META', s: 'operating-income', freq: 'A', statement: 'income-statement' }}\"><i style='font-size:18px; color:#337ab7;' class='fas fa-chart-bar'><\/i><\/span><\/div>","2021-12-31":"46753.00000","2020-12-31":"32671.00000","2019-12-31":"23986.00000","2018-12-31":"24913.00000","2017-12-31":"20203.00000","2016-12-31":"12427.00000","2015-12-31":"6225.00000","2014-12-31":"4994.00000","2013-12-31":"2804.00000","2012-12-31":"538.00000","2011-12-31":"1756.00000","2010-12-31":"1032.00000","2009-12-31":"262.00000"},{"field_name":"<a href='\/stocks\/charts\/META\/meta-platforms\/total-non-operating-income-expense'>Total Non-Operating Income\/Expense<\/a>","popup_icon":"<div class='ajax-chart' data-tipped-options=\"ajax: {data: { t: 'META', s: 'total-non-operating-income-expense', freq: 'A', statement: 'income-statement' }}\"><i style='font-size:18px; color:#337ab7;' class='fas fa-chart-bar'><\/i><\/span><\/div>","2021-12-31":"531.00000","2020-12-31":"509.00000","2019-12-31":"826.00000","2018-12-31":"448.00000","2017-12-31":"391.00000","2016-12-31":"91.00000","2015-12-31":"-31.00000","2014-12-31":"-84.00000","2013-12-31":"-50.00000","2012-12-31":"-44.00000","2011-12-31":"-61.00000","2010-12-31":"-24.00000","2009-12-31":"-8.00000"},{"field_name":"<a href='\/stocks\/charts\/META\/meta-platforms\/pre-tax-income'>Pre-Tax Income<\/a>","popup_icon":"<div class='ajax-chart' data-tipped-options=\"ajax: {data: { t: 'META', s: 'pre-tax-income', freq: 'A', statement: 'income-statement' }}\"><i style='font-size:18px; color:#337ab7;' class='fas fa-chart-bar'><\/i><\/span><\/div>","2021-12-31":"47284.00000","2020-12-31":"33180.00000","2019-12-31":"24812.00000","2018-12-31":"25361.00000","2017-12-31":"20594.00000","2016-12-31":"12518.00000","2015-12-31":"6194.00000","2014-12-31":"4910.00000","2013-12-31":"2754.00000","2012-12-31":"494.00000","2011-12-31":"1695.00000","2010-12-31":"1008.00000","2009-12-31":"254.00000"},{"field_name":"<a href='\/stocks\/charts\/META\/meta-platforms\/total-provision-income-taxes'>Income Taxes<\/a>","popup_icon":"<div class='ajax-chart' data-tipped-options=\"ajax: {data: { t: 'META', s: 'total-provision-income-taxes', freq: 'A', statement: 'income-statement' }}\"><i style='font-size:18px; color:#337ab7;' class='fas fa-chart-bar'><\/i><\/span><\/div>","2021-12-31":"7914.00000","2020-12-31":"4034.00000","2019-12-31":"6327.00000","2018-12-31":"3249.00000","2017-12-31":"4660.00000","2016-12-31":"2301.00000","2015-12-31":"2506.00000","2014-12-31":"1970.00000","2013-12-31":"1254.00000","2012-12-31":"441.00000","2011-12-31":"695.00000","2010-12-31":"402.00000","2009-12-31":"25.00000"},{"field_name":"<a href='\/stocks\/charts\/META\/meta-platforms\/income-after-taxes'>Income After Taxes<\/a>","popup_icon":"<div class='ajax-chart' data-tipped-options=\"ajax: {data: { t: 'META', s: 'income-after-taxes', freq: 'A', statement: 'income-statement' }}\"><i style='font-size:18px; color:#337ab7;' class='fas fa-chart-bar'><\/i><\/span><\/div>","2021-12-31":"39370.00000","2020-12-31":"29146.00000","2019-12-31":"18485.00000","2018-12-31":"22112.00000","2017-12-31":"15934.00000","2016-12-31":"10217.00000","2015-12-31":"3688.00000","2014-12-31":"2940.00000","2013-12-31":"1500.00000","2012-12-31":"53.00000","2011-12-31":"1000.00000","2010-12-31":"606.00000","2009-12-31":"229.00000"},{"field_name":"<span style='color:#337ab7;'>Other Income<\/span>","popup_icon":"","2021-12-31":"","2020-12-31":"","2019-12-31":"","2018-12-31":"","2017-12-31":"","2016-12-31":"","2015-12-31":"","2014-12-31":"","2013-12-31":"","2012-12-31":"","2011-12-31":"","2010-12-31":"","2009-12-31":""},{"field_name":"<a href='\/stocks\/charts\/META\/meta-platforms\/income-from-continuous-operations'>Income From Continuous Operations<\/a>","popup_icon":"<div class='ajax-chart' data-tipped-options=\"ajax: {data: { t: 'META', s: 'income-from-continuous-operations', freq: 'A', statement: 'income-statement' }}\"><i style='font-size:18px; color:#337ab7;' class='fas fa-chart-bar'><\/i><\/span><\/div>","2021-12-31":"39370.00000","2020-12-31":"29146.00000","2019-12-31":"18485.00000","2018-12-31":"22112.00000","2017-12-31":"15934.00000","2016-12-31":"10217.00000","2015-12-31":"3688.00000","2014-12-31":"2940.00000","2013-12-31":"1500.00000","2012-12-31":"53.00000","2011-12-31":"1000.00000","2010-12-31":"606.00000","2009-12-31":"229.00000"},{"field_name":"<span style='color:#337ab7;'>Income From Discontinued Operations<\/span>","popup_icon":"","2021-12-31":"","2020-12-31":"","2019-12-31":"","2018-12-31":"","2017-12-31":"","2016-12-31":"","2015-12-31":"","2014-12-31":"","2013-12-31":"","2012-12-31":"","2011-12-31":"","2010-12-31":"","2009-12-31":""},{"field_name":"<a href='\/stocks\/charts\/META\/meta-platforms\/net-income'>Net Income<\/a>","popup_icon":"<div class='ajax-chart' data-tipped-options=\"ajax: {data: { t: 'META', s: 'net-income', freq: 'A', statement: 'income-statement' }}\"><i style='font-size:18px; color:#337ab7;' class='fas fa-chart-bar'><\/i><\/span><\/div>","2021-12-31":"39370.00000","2020-12-31":"29146.00000","2019-12-31":"18485.00000","2018-12-31":"22111.00000","2017-12-31":"15920.00000","2016-12-31":"10188.00000","2015-12-31":"3669.00000","2014-12-31":"2925.00000","2013-12-31":"1491.00000","2012-12-31":"32.00000","2011-12-31":"668.00000","2010-12-31":"372.00000","2009-12-31":"122.00000"},{"field_name":"<a href='\/stocks\/charts\/META\/meta-platforms\/ebitda'>EBITDA<\/a>","popup_icon":"<div class='ajax-chart' data-tipped-options=\"ajax: {data: { t: 'META', s: 'ebitda', freq: 'A', statement: 'income-statement' }}\"><i style='font-size:18px; color:#337ab7;' class='fas fa-chart-bar'><\/i><\/span><\/div>","2021-12-31":"54720.00000","2020-12-31":"39533.00000","2019-12-31":"29727.00000","2018-12-31":"29228.00000","2017-12-31":"23228.00000","2016-12-31":"14769.00000","2015-12-31":"8170.00000","2014-12-31":"6237.00000","2013-12-31":"3815.00000","2012-12-31":"1187.00000","2011-12-31":"2079.00000","2010-12-31":"1171.00000","2009-12-31":"340.00000"},{"field_name":"<a href='\/stocks\/charts\/META\/meta-platforms\/ebit'>EBIT<\/a>","popup_icon":"<div class='ajax-chart' data-tipped-options=\"ajax: {data: { t: 'META', s: 'ebit', freq: 'A', statement: 'income-statement' }}\"><i style='font-size:18px; color:#337ab7;' class='fas fa-chart-bar'><\/i><\/span><\/div>","2021-12-31":"46753.00000","2020-12-31":"32671.00000","2019-12-31":"23986.00000","2018-12-31":"24913.00000","2017-12-31":"20203.00000","2016-12-31":"12427.00000","2015-12-31":"6225.00000","2014-12-31":"4994.00000","2013-12-31":"2804.00000","2012-12-31":"538.00000","2011-12-31":"1756.00000","2010-12-31":"1032.00000","2009-12-31":"262.00000"},{"field_name":"<a href='\/stocks\/charts\/META\/meta-platforms\/basic-shares-outstanding'>Basic Shares Outstanding<\/a>","popup_icon":"<div class='ajax-chart' data-tipped-options=\"ajax: {data: { t: 'META', s: 'basic-shares-outstanding', freq: 'A', statement: 'income-statement' }}\"><i style='font-size:18px; color:#337ab7;' class='fas fa-chart-bar'><\/i><\/span><\/div>","2021-12-31":"2815.00000","2020-12-31":"2851.00000","2019-12-31":"2854.00000","2018-12-31":"2890.00000","2017-12-31":"2901.00000","2016-12-31":"2863.00000","2015-12-31":"2803.00000","2014-12-31":"2614.00000","2013-12-31":"2420.00000","2012-12-31":"2006.00000","2011-12-31":"1294.00000","2010-12-31":"1107.00000","2009-12-31":"1020.00000"},{"field_name":"<a href='\/stocks\/charts\/META\/meta-platforms\/shares-outstanding'>Shares Outstanding<\/a>","popup_icon":"<div class='ajax-chart' data-tipped-options=\"ajax: {data: { t: 'META', s: 'shares-outstanding', freq: 'A', statement: 'income-statement' }}\"><i style='font-size:18px; color:#337ab7;' class='fas fa-chart-bar'><\/i><\/span><\/div>","2021-12-31":"2859.00000","2020-12-31":"2888.00000","2019-12-31":"2876.00000","2018-12-31":"2921.00000","2017-12-31":"2956.00000","2016-12-31":"2925.00000","2015-12-31":"2853.00000","2014-12-31":"2664.00000","2013-12-31":"2517.00000","2012-12-31":"2166.00000","2011-12-31":"1508.00000","2010-12-31":"1414.00000","2009-12-31":"2732.00000"},{"field_name":"<a href='\/stocks\/charts\/META\/meta-platforms\/eps-basic-net-earnings-per-share'>Basic EPS<\/a>","popup_icon":"<div class='ajax-chart' data-tipped-options=\"ajax: {data: { t: 'META', s: 'eps-basic-net-earnings-per-share', freq: 'A', statement: 'income-statement' }}\"><i style='font-size:18px; color:#337ab7;' class='fas fa-chart-bar'><\/i><\/span><\/div>","2021-12-31":"13.99000","2020-12-31":"10.22000","2019-12-31":"6.48000","2018-12-31":"7.65000","2017-12-31":"5.49000","2016-12-31":"3.56000","2015-12-31":"1.31000","2014-12-31":"1.12000","2013-12-31":"0.62000","2012-12-31":"0.02000","2011-12-31":"0.52000","2010-12-31":"0.34000","2009-12-31":"0.12000"},{"field_name":"<a href='\/stocks\/charts\/META\/meta-platforms\/eps-earnings-per-share-diluted'>EPS - Earnings Per Share<\/a>","popup_icon":"<div class='ajax-chart' data-tipped-options=\"ajax: {data: { t: 'META', s: 'eps-earnings-per-share-diluted', freq: 'A', statement: 'income-statement' }}\"><i style='font-size:18px; color:#337ab7;' class='fas fa-chart-bar'><\/i><\/span><\/div>","2021-12-31":"13.77000","2020-12-31":"10.09000","2019-12-31":"6.43000","2018-12-31":"7.57000","2017-12-31":"5.39000","2016-12-31":"3.49000","2015-12-31":"1.29000","2014-12-31":"1.10000","2013-12-31":"0.60000","2012-12-31":"0.01000","2011-12-31":"0.46000","2010-12-31":"0.28000","2009-12-31":"0.10000"}]

In [35]:
script_text

[{'field_name': "<a href='\\/stocks\\/charts\\/META\\/meta-platforms\\/revenue'>Revenue<\\/a>",
  'popup_icon': '<div class=\'ajax-chart\' data-tipped-options="ajax: {data: { t: \'META\', s: \'revenue\', freq: \'A\', statement: \'income-statement\' }}"><i style=\'font-size:18px; color:#337ab7;\' class=\'fas fa-chart-bar\'><\\/i><\\/span><\\/div>',
  '2021-12-31': '117929.00000',
  '2020-12-31': '85965.00000',
  '2019-12-31': '70697.00000',
  '2018-12-31': '55838.00000',
  '2017-12-31': '40653.00000',
  '2016-12-31': '27638.00000',
  '2015-12-31': '17928.00000',
  '2014-12-31': '12466.00000',
  '2013-12-31': '7872.00000',
  '2012-12-31': '5089.00000',
  '2011-12-31': '3711.00000',
  '2010-12-31': '1974.00000',
  '2009-12-31': '777.00000'},
 {'field_name': "<a href='\\/stocks\\/charts\\/META\\/meta-platforms\\/cost-goods-sold'>Cost Of Goods Sold<\\/a>",
  'popup_icon': '<div class=\'ajax-chart\' data-tipped-options="ajax: {data: { t: \'META\', s: \'cost-goods-sold\', freq: \'A\', stateme

In [22]:
#import json
#data = json.loads('{"one" : "1", "two" : "2", "three" : "3"}')


In [23]:
souped_html

<!DOCTYPE html>
<!--[if lt IE 7]>      <html class="no-js lt-ie9 lt-ie8 lt-ie7"> <![endif]--><!--[if IE 7]>         <html class="no-js lt-ie9 lt-ie8"> <![endif]--><!--[if IE 8]>         <html class="no-js lt-ie9"> <![endif]--><!--[if gt IE 8]><!--><html class="no-js"><!--<![endif]--><head>
        <meta charset="utf-8"/>
        <meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
		<link href="https://www.macrotrends.net/stocks/charts/META/meta-platforms/income-statement" rel="canonical"/>
		<title>Meta Platforms Income Statement 2009-2022 | META | MacroTrends</title>
		<meta content="Ten years of annual and quarterly income statements for Meta Platforms (META). The income statement summarizes the revenues, expenses and profit generated by a business over an annual or quarterly period." name="description"/>
		<meta content="" name="robots"/>
				
		<link href="/assets/images/icons/FAVICON/macro-trends_favicon.ico" rel="shortcut icon" type="image/x-icon"/>

		<meta content="